In [ ]:
# import findspark
import findspark
# initialize findspark with spark directory
findspark.init("C:\Program Files\Spark\spark-3.3.1-bin-hadoop3")
#findspark.init("/Users/wouterdewitte/spark/")
# import pyspark
import pyspark
# create spark context
sc = pyspark.SparkContext()
# create spark session 
spark = pyspark.sql.SparkSession(sc)

In [ ]:
# import packages
import os 
import pickle
import re
from datetime import datetime
import requests
import pytz
import emojis
import pandas as pd
import numpy as np
import ast
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import array_contains
import matplotlib.pyplot as plt 

## General

In this notebook we will buid a model that predicts if the trend of a certain topic goes up or down on a certain day based on Twitter data of that day.

## 1. Import Data

### 1.1 Google Trends

In [ ]:
# read trend data 
trend = spark.read.csv(".././../data/Google_trends/daily_trends.csv", header=True, inferSchema=True, sep=';')

In [ ]:
trend

In [ ]:
from pyspark.sql.window import Window

w = Window().partitionBy().orderBy(col("date"))
trend.withColumn("dependent_vegan", lag("dependent_vegan", -1, 0).over(w)).show()

In [ ]:
# create SQL view
trend.createOrReplaceTempView("trendSQL")

The binary variable indicates if the trend goes up or down.

### 1.2 Twitter

In [ ]:
# define data dir
data_dir = "../../data/Topic/"

# get all twitter files
tweet_files = [os.path.join(data_dir, obs) for obs in os.listdir(data_dir)] 

In [ ]:
# import twitter data 
#twitter_df = spark.read.json(tweet_files)

In [ ]:
list_hashtags = ["vegan"]

data_dir = ".././../data/Topic/"
tweet_files = [os.path.join(data_dir, obs) for obs in os.listdir(data_dir)]
files_hashtags = [file for file in tweet_files if (file.find(list_hashtags[0]) != -1)]             
twitter_df = spark.read.option("multiline","true").json(files_hashtags) 
twitter_df.count()

In [ ]:
# select interesting features
twitter_df = twitter_df.select(F.col('user.name'),
                                F.col('user.screen_name'),
                                F.col('user.followers_count'),
                                F.col('user.following'),
                                F.col('user.statuses_count'),
                                F.col('user.listed_count'),
                                F.col('created_at'),
                                F.col('full_text'),
                                F.col('entities.hashtags'),
                                F.col('favorite_count'),
                                F.col('retweet_count'),
                                F.col('user.friends_count'))

## 2. Data Preprocessing

### 2.1 Check time period

In [ ]:
# function to convert Twitter date string format
def getDate(date):
    if date is not None:
        return str(datetime.strptime(date,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).strftime("%Y-%m-%d %H:%M:%S"))
    else:
        return None

# UDF declaration
date_udf = F.udf(getDate, StringType())

# apply udf
twitter_df = twitter_df.withColumn('post_created_at', F.to_utc_timestamp(date_udf("created_at"), "UTC"))

In [ ]:
# get first post
first_post = F.min('post_created_at').alias('earliest')
# get latest post
latest_post = F.max('post_created_at').alias('latest')
# show tweet period in our dataset
twitter_df.select(first_post, latest_post).show()

### 2.2 Remove retweets and duplicates

In [ ]:
# drop all retweets from dataset
no_retweets_df = twitter_df.filter(~F.col("full_text").startswith("RT"))

In [ ]:
# first sort no_retweets_df based on date in chronological order (most recent ones on top)
no_retweets_sorted_df = no_retweets_df.sort("post_created_at", ascending=False)

In [ ]:
# number of observations before dropping duplicates
no_retweets_sorted_df.count()

In [ ]:
# drop duplicates based on tweet text and the profile it was posted from
final_no_duplicates_df = no_retweets_sorted_df.drop_duplicates(["full_text", "screen_name"])

In [ ]:
# number of observations after dropping duplicates
final_no_duplicates_df.count()

In [ ]:
# rename dataframe
final_twitter_df = final_no_duplicates_df

## 3. Independent Variables

For our independent variables we need to design a pipeline that transforms the data into the desired aggregated metrics per day.

### 3.0 Feature Engineering

#### Define Functions

In [ ]:
# define function to count hashtags
def get_hashtags(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "#" in word:
            counter += 1
    return(counter)

In [ ]:
# define function to count mentions
def get_mentions(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "@" in word:
            counter += 1
    return(counter)

In [ ]:
# define function to count exclamation marks
def get_exclamation_marks(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "!" in word:
            counter += 1
    return(counter)

In [ ]:
# define function to count number of emojis used
def emoji_counter(text):
    nr_emojis = emojis.count(text)
    return(nr_emojis)

In [ ]:
# define function to calculate engagement rate
def engagement_rate(favorite_count, retweet_count, followers_count):
    if(followers_count == 0):
        eng_rate = 0
    else:
        eng_rate = (favorite_count + retweet_count)/followers_count
    
    return eng_rate

In [ ]:
# register functions as udf
get_hashtags_UDF = F.udf(get_hashtags, IntegerType())
get_mentions_UDF = F.udf(get_mentions, IntegerType())
get_exclamation_marks_UDF = F.udf(get_exclamation_marks, IntegerType())
emoji_counter_UDF = F.udf(emoji_counter, IntegerType())
engagement_rate_UDF = F.udf(engagement_rate, DoubleType())

In [ ]:
# apply functions to create new features
final_twitter_df = final_twitter_df.withColumn("emoji_count", emoji_counter_UDF("full_text")) \
        .withColumn("text_tokenized", F.split("full_text", " ")) \
        .withColumn("num_words", F.size("text_tokenized")) \
        .withColumn("num_hashtags", get_hashtags_UDF("text_tokenized")) \
        .withColumn("num_mentions", get_mentions_UDF("text_tokenized")) \
        .withColumn("num_exclamation_marks", get_exclamation_marks_UDF("text_tokenized")) \
        .withColumn("engagement_rate", engagement_rate_UDF("favorite_count", "retweet_count", "followers_count"))

#### Text Cleaning

In [ ]:
# filter for english tweets (NOTE: for the assignment you can translate non-english tweets using an API)
final_twitter_df = final_twitter_df.filter(F.col("lang") == "en")

In [ ]:
# check number of observations
final_twitter_df.count()

In [ ]:
# define function to clean text
def clean_text(string):
    
    # define numbers
    NUMBERS = '0123456789'
    PUNCT = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    
    # convert text to lower case
    cleaned_string = string.lower()
    
    # remove URLS
    cleaned_string = re.sub(r'http\S+', ' ', cleaned_string)
    
    # replace emojis by words
    cleaned_string = emojis.decode(cleaned_string)
    cleaned_string = cleaned_string.replace(":"," ").replace("_"," ")
    cleaned_string = ' '.join(cleaned_string.split())
    
    # remove numbers
    cleaned_string = "".join([char for char in cleaned_string if char not in NUMBERS])
    
    # remove punctuation
    cleaned_string = "".join([char for char in cleaned_string if char not in PUNCT])
    
    # remove words conisting of one character (or less)
    cleaned_string = ' '.join([w for w in cleaned_string.split() if len(w) > 1])
    
    # return
    return(cleaned_string)

In [ ]:
# convert to udf
clean_text_udf = F.udf(clean_text, StringType())

In [ ]:
# clean string
final_twitter_df = final_twitter_df.withColumn("cleaned_text", clean_text_udf(F.col("full_text")))

In [ ]:
# check
pd.set_option('display.max_colwidth', None)
final_twitter_df.select("full_text", "cleaned_text").limit(5).toPandas()

#### Sentiment

VADER sentimental analysis relies on a dictionary that maps lexical features to emotion intensities known as sentiment scores. The sentiment score of a text can be obtained by summing up the intensity of each word in the text.

In [ ]:
#using the vaderSentiment package 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [ ]:
# define the function to extract the sentiment
def get_sentiment(sentence):

    # initialize sentiment analyzer
    sid_obj = SentimentIntensityAnalyzer()

    # get sentiment dict
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    # get positive sentiment score
    pos_sentiment = sentiment_dict["pos"]
    
    # return positive sentiment score
    return(pos_sentiment)

get_sentiment_udf = udf(get_sentiment, DoubleType())

In [ ]:
final_twitter_df = final_twitter_df.withColumn("sentiment_vader", get_sentiment_udf(F.col("cleaned_text")))

In [ ]:
final_twitter_df.select("cleaned_text", "sentiment_vader").show()

#### TextBlob

TextBlob returns polarity and subjectivity of a sentence. 

**Polarity** lies between [-1,1],  -1 defines a negative sentiment and 1 defines a positive sentiment.  

**Subjectivity** quantifies the amount of personal opinion and factual information contained in the text. Subjectivity lies between [0,1]. The higher subjectivity means that the text contains personal opinion rather than factual information. 

In [ ]:
#use polarity and subjectivity from TextBlob 
#https://textblob.readthedocs.io/en/dev/
from textblob import TextBlob

# define function to get polarity score of text document
def get_polarity(row):
    textBlob_review = TextBlob(row)
    return textBlob_review.sentiment[0]
# define function to get subjectivity score of text document
def get_subjectivity(row):
    textBlob_review = TextBlob(row)
    return textBlob_review.sentiment[1]
get_polarity_udf = F.udf(get_polarity, DoubleType())
get_subjectivity_udf = F.udf(get_subjectivity, DoubleType())

final_twitter_df = final_twitter_df.withColumn('polarity', get_polarity_udf(F.col('cleaned_text')))\
        .withColumn('subjectivity', get_subjectivity_udf(F.col('cleaned_text')))

final_twitter_df.select("cleaned_text", "sentiment_vader", "polarity", "subjectivity").show()

In [ ]:
final_twitter_df = final_twitter_df.withColumn('polarity', get_polarity_udf(F.col('cleaned_text')))\
        .withColumn('subjectivity', get_subjectivity_udf(F.col('cleaned_text')))

In [ ]:
final_twitter_df.select("cleaned_text", "sentiment_vader", "polarity", "subjectivity").show()

In [ ]:
# create SQL view
final_twitter_df.createOrReplaceTempView("twitterSQL")

### 3.1 Volume of tweets 

In [ ]:
# select the relevant data
volume = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, COUNT(*) as volume \
                                    FROM twitterSQL \
                                    GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                                    ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show 
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
volume.show(100)

In [ ]:
# create SQL view
volume.createOrReplaceTempView("volumeSQL")

### 3.2 Average likes

We exclude tweets with 0 likes.

In [ ]:
# select the relevant data
avg_likes = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(favorite_count) as avg_likes \
                           FROM twitterSQL \
                           WHERE favorite_count > 0 \
                           GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                           ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show 
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_likes.show()

In [ ]:
# create SQL view
avg_likes.createOrReplaceTempView("avg_likesSQL")

### 3.3 Average Retweets

We exclude tweets with 0 retweets.

In [ ]:
# select the relevant data
avg_retweets = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(retweet_count) as avg_retweets \
                          FROM twitterSQL \
                          WHERE retweet_count > 0 \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show 
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_retweets.show()

In [ ]:
# create SQL view
avg_retweets.createOrReplaceTempView("avg_retweetsSQL")

### 3.4 Average Engagement rate

We define engagement rate of a tweet as the sum of likes and retweets divided by the amount of followers of the account that sent out the tweet. For our purpose we will take the avergage engagement rate per day. We exclude accounts who have no followers and we only take tweets into account which are liked and retweeted at least once.

In [ ]:
# select the relevant data
avg_engagement_rate = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(engagement_rate) as avg_engagement_rate \
                                     FROM twitterSQL \
                                     GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                                     ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_engagement_rate.show()

In [ ]:
# create SQL view
avg_engagement_rate.createOrReplaceTempView("avg_engagement_rateSQL")

### 3.5 Number of influencers

We will calculate how many influencers actively tweeted a certain day. We define an influencer as someone with:
- followers > 1000 
- engagement_rate > 0.20 
- weekly tweet frequency > 5

In [ ]:
def get_influencers(follower_count_tresh, eng_rate_tresh, freq_week_tresh, data):

    #df
    df = data
    
    # get all users with their amount of followers
    influencers = df.groupBy("screen_name") \
                    .agg(first("followers_count").alias("followers_count"))

    # average engagement rate for each user
    eng_rate = df.withColumn('eng_rate', ((df['favorite_count'] + df['retweet_count'])/df['followers_count']))

    eng_rate_user = eng_rate.groupBy("screen_name") \
                            .agg(avg("eng_rate").alias("eng_rate"))

    # average freq_weekly per user
    freq_week = df.withColumn("year", year(df["post_created_at"]))
    freq_week = freq_week.withColumn('week', weekofyear('post_created_at'))

    freq_week = freq_week.groupBy('screen_name', 'year', 'week').agg(countDistinct("full_text"))\
                    .withColumnRenamed("count(full_text)", "freq") \
                        .sort('screen_name', 'year', 'week', ascending = True)
    freq_week = freq_week.select('screen_name', 'freq')

    freq_week = freq_week.groupby("screen_name").agg(avg(freq_week.freq).alias('freq'))

    # put the data together
    data_joined = eng_rate_user.join(influencers, "screen_name").join(freq_week, "screen_name")

    # filter the data
    data_joined = data_joined.filter((data_joined.followers_count > follower_count_tresh) & (data_joined.eng_rate > eng_rate_tresh) & (data_joined.freq > freq_week_tresh))
    
    # show the data
    data_joined.show()
    return data_joined

In [ ]:
influencers = get_influencers(1000, 0.002, 2, final_twitter_df)

In [ ]:
# create SQL view
influencers.createOrReplaceTempView("influencersSQL")

In [ ]:
# select the relevant data
number_of_influencers = spark.sql(" SELECT DATE_FORMAT(a.post_created_at, 'Y-M-dd') as date, COUNT(b.screen_name) as influencers \
                                    FROM twitterSQL a \
                                    RIGHT OUTER JOIN influencersSQL b ON a.screen_name = b.screen_name\
                                    GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                                    ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
number_of_influencers.show()

In [ ]:
# create SQL view
number_of_influencers.createOrReplaceTempView("number_of_influencersSQL")

### 3.6 Average Followers 

In [ ]:
# select the relevant data
avg_followers = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(followers_count) as avg_followers \
                          FROM twitterSQL \
                          WHERE followers_count > 0 \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_followers.show()

In [ ]:
# create SQL view
number_of_influencers.createOrReplaceTempView("followersSQL")

### 3.7 Average Emoji Counts

In [ ]:
# select the relevant data
avg_emoji = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(emoji_count) as avg_emojis \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_emoji.show()

In [ ]:
# create SQL view
number_of_influencers.createOrReplaceTempView("emojiSQL")

### 3.8 Avergae Number of Words

In [ ]:
# select the relevant data
avg_words = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(num_words) as avg_words \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_words.show()

In [ ]:
# create SQL view
avg_words.createOrReplaceTempView("wordsSQL")

### 3.9 Avergae Number of Hashtags

In [ ]:
# select the relevant data
avg_hashtags = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(num_hashtags) as avg_hashtags \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_hashtags.show()

In [ ]:
# create SQL view
avg_hashtags.createOrReplaceTempView("hashtagsSQL")

### 3.10 Average Number of Mentions

In [ ]:
# select the relevant data
avg_mentions = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(num_mentions) as avg_mentions \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_mentions.show()

In [ ]:
# create SQL view
avg_mentions.createOrReplaceTempView("mentionsSQL")

### 3.11 Average Number of Exclamation Marks

In [ ]:
# select the relevant data
avg_marks = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(num_exclamation_marks) as avg_exclamation_marks \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_marks.show()

In [ ]:
# create SQL view
avg_marks.createOrReplaceTempView("marksSQL")

### 3.12 Sentiment 

In [ ]:
# select the relevant data
avg_sentiment = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(sentiment_vader) as avg_sentiment \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_sentiment.show()

In [ ]:
# create SQL view
avg_sentiment.createOrReplaceTempView("sentimentSQL")

### 3.13 Polarity

In [ ]:
# select the relevant data
avg_polarity = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(polarity) as avg_polarity \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_polarity.show()

In [ ]:
# create SQL view
avg_polarity.createOrReplaceTempView("polaritySQL")

### 3.13 Subjectivity

In [ ]:
# select the relevant data
avg_subjectivity = spark.sql("SELECT DATE_FORMAT(post_created_at, 'Y-M-dd') as date, AVG(subjectivity) as avg_subjectivity \
                          FROM twitterSQL \
                          GROUP BY DATE_FORMAT(post_created_at, 'Y-M-dd') \
                          ORDER BY DATE_FORMAT(post_created_at, 'Y-M-dd')")

In [ ]:
# show
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
avg_subjectivity.show()

In [ ]:
# create SQL view
avg_subjectivity.createOrReplaceTempView("subjectivitySQL")

## 4. Basetable

In [ ]:
# create basetable
basetable = spark.sql("SELECT DATE_FORMAT(a.date, 'Y-M-dd') as date, a.dependent_vegan, b.tweet_volume, COALESCE(c.avg_likes,0) as avg_likes, \
                       COALESCE(d.avg_retweets,0) as avg_retweets, \
                       COALESCE(e.avg_engagement_rate,0) as avg_engagement_rate, COALESCE(f.influencers,0) as influencers \
                       FROM trendSQL a \
                       INNER JOIN tweet_volumeSQL b ON DATE_FORMAT(a.date, 'Y-M-dd') = b.date \
                       LEFT OUTER JOIN avg_likesSQL c ON b.date = c.date \
                       LEFT OUTER JOIN avg_retweetsSQL d ON c.date = d.date \
                       LEFT OUTER JOIN avg_engagement_rateSQL e ON d.date = e.date \
                       LEFT OUTER JOIN number_of_influencersSQL f ON e.date = f.date \
                       LEFT OUTER JOIN followersSQL g ON f.date = g.date \ 
                       LEFT OUTER JOIN emojiSQL h ON g.date = h.date \
                       LEFT OUTER JOIN wordsSQL i ON h.date = i.date \
                       LEFT OUTER JOIN hashtagsSQL j ON i.date = j.date \
                       LEFT OUTER JOIN mentionsSQL k ON j.date = k.date \
                       LEFT OUTER JOIN marksSQL l ON k.date = l.date \
                       LEFt OUTER JOIN sentimentSQL m ON l.date = m.date \
                       LEFT OUTER JOIN polaritySQL n ON m.date = n.date \
                       LEFT OUTER JOIN subjectivitySQL o ON n.date = o.date")

In [ ]:
# show
basetable.show(50)

In [ ]:
# Dropping the date column as this is not a feature 
basetable.drop('date', axis=1, inplace=True)
basetable.head()

In [ ]:
# store basetable as a .parquet file
basetable.to_parquet("./../../data/basetable_vegan_trend_prediction.parquet")

In [ ]:
# export basetable as a .json file
basetable.to_json("./../../data/basetable_vegan_trend_prediction.json", orient="records", force_ascii=False, lines=True)

## 5. Modelling

In [ ]:
# read in the saved basetable (.parquet)
#basetable_df = spark.read.parquet("./../../data/basetable_vegan_trend_prediction.parquet")

In [ ]:
# read in the saved basetable (.json)
#basetable_df = spark.read.json("./../../data/basetable_vegan_trend_prediction.json")

In [ ]:
#basetable_df.toPandas().head()

In [ ]:
# import the required functions
from pyspark.ml.feature import Binarizer, StringIndexer, VectorIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.types import DoubleType

In [ ]:
# define string indexer to index price 
SI = StringIndexer(inputCol = 'dependent_vegan', outputCol = 'label')

# define vector assembler for numeric variables
numColumns = ['volume','avg_likes','avg_retweets','avg_engagement_rate','influencers' \
             ,'avg_followers', 'avg_emojis', 'avg_words', 'avg_hashtags', 'avg_mentions' \
             ,'avg_exclamation_marks', 'avg_sentiment', 'avg_polarity','avg_subjectivity']
VAnum = VectorAssembler(inputCols=numColumns, outputCol="numFeatures")

In [ ]:
# define pipeline stages
stages = [SI, VAnum]
# define pipeline and fit on data
preprocessingPipeline = Pipeline().setStages(stages).fit(basetable)
# apply pipeline on data
basetable_df = preprocessingPipeline.transform(basetable)

In [ ]:
# select features and labels
basetable = basetable.select(["numFeatures", "label"])

In [ ]:
# check
basetable.show(5)

**Logistic Regression**
- Split the data in a train and test set (70/30).
- Build one pipeline that:
  - standardizes the numerical variables
  - applies a logistic regression to the data
  - check the performance using the AUC.

We cannot use the randomsplit function, because we have time series data, so we have to use another approach

First we look at the amount of observations that will be assigned to the training set 

In [ ]:
nr_train = int(basetable.count()*0.7)
nr_train

convert the final basetable to a pandas dataset 

In [ ]:
basetable_pd = basetable.toPandas()
basetable_pd.head()

Split the dataframe into train and test 

In [ ]:
train_pd = basetable_pd.iloc[:nr_train,:]
test_pd = basetable_pd.iloc[nr_train:,:]
train = spark.createDataFrame(train_pd)
test = spark.createDataFrame(test_pd)

In [ ]:
# check number of observations in train and test set
print(train.count())
print(test.count())

In [ ]:
# inspect distribution of label in train and test set
basetable.groupBy("label").count().show()
train.groupBy("label").count().show()
test.groupBy("label").count().show()

In [ ]:
# import required features
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
# define scaler
SS = StandardScaler(inputCol = 'numFeatures', outputCol = 'scaledNumFeatures', withStd = True, withMean = False)

# define vector assembler
VA = VectorAssembler(inputCols = ['scaledNumFeatures'], outputCol = 'features')

# define logistic regression model
LR = LogisticRegression(labelCol = 'label', featuresCol = 'features', maxIter = 10)

In [ ]:
# define pipeline stages
stages = [SS, VA, LR]
# create pipeline and fit on training set
lrModelPipeline = Pipeline().setStages(stages).fit(train)
# apply pipeline on test set to get predictions
predictions = lrModelPipeline.transform(test)

In [ ]:
# inspect predictions
predictions.show(5)

In [ ]:
# define evaluator
evaluator = BinaryClassificationEvaluator()
# get evaluation metric
lrAUC = evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderROC'})
# inspect model performance
print('AUC lr: %f' %(lrAUC))